pmgus.ipynb
"Pre-Market Gap-Up Screener"

In [6]:
import pandas as pd
import numpy as np

In [8]:
# load list from TradingView
""" 
Market : US
Exchange : NASDAQ, NYSE
Pre-market Chg > $0.00 USD

Stocks = ~1782
"""
base_file_path = '/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/'
file_name =  'tv_screen_gap-up_2024-10-28.csv'
pmgus_df = pd.read_csv(base_file_path + file_name)


print(len(pmgus_df))
pmgus_df.tail(2)

1782


,Symbol,Description,Industry,Sector,Exchange,Index,Market capitalization,Market capitalization - Currency,Price,Price - Currency,...,"Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",International Securities Identification Number,Relative Volume 1 day.1,Recent earnings date,Upcoming earnings date,Analyst Rating,Target price 1 year,Target price 1 year - Currency,Technical Rating 5 minutes
1780,SPAI,Safe Pro Group Inc.,Aerospace & defense,Electronic technology,NASDAQ,"NASDAQ Composite, NASDAQ Industrials",NaN,NaN,2.21,USD,...,2.191130,2.138519,US78642D1019,2.807405,NaN,NaN,NaN,NaN,NaN,Buy
1781,PDO,PIMCO Dynamic Income Opportunities Fund,Investment trusts/Mutual funds,Miscellaneous,NYSE,NaN,NaN,NaN,13.69,USD,...,13.696585,13.648606,US69355M1071,1.171612,NaN,NaN,NaN,NaN,NaN,Sell


In [9]:
def categorize_market_cap(df):
    """Categorize stocks based on market capitalization."""
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    conditions = [
        (df['Market capitalization'] >= 200_000_000_000),  # Titans
        (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
        (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
        (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
        (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
        (df['Market capitalization'] <= 50_000_000)  # Shrimp
    ]
    categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    # drop Undefined
    df = df[df['marketCapType'] != 'Undefined']
    # convert to numeric the Market capitalization column
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'])
    
    return df

In [14]:
# categorize market cap
pmgus_df = categorize_market_cap(pmgus_df).copy()


print(len(pmgus_df))
pmgus_df.head()

1769


,Symbol,Description,Industry,Sector,Exchange,Index,Market capitalization,Market capitalization - Currency,Price,Price - Currency,...,"Bollinger Bands (20) 5 minutes, Lower",International Securities Identification Number,Relative Volume 1 day.1,Recent earnings date,Upcoming earnings date,Analyst Rating,Target price 1 year,Target price 1 year - Currency,Technical Rating 5 minutes,marketCapType
0,AAPL,Apple Inc.,Telecommunications equipment,Electronic technology,NASDAQ,"S&P 500, NASDAQ 100, Dow Jones Industrial Aver...",3.548646e+12,USD,233.40,USD,...,233.532292,US0378331005,0.868639,2024-08-01,2024-10-31,Buy,244.122439,USD,Sell,Titans
1,NVDA,NVIDIA Corporation,Semiconductors,Electronic technology,NASDAQ,"S&P 500, NASDAQ 100, NASDAQ Composite, PHLX Se...",3.446955e+12,USD,140.52,USD,...,140.748566,US67066G1040,0.691049,2024-08-28,2024-11-20,Strong buy,150.958699,USD,Strong sell,Titans
2,MSFT,Microsoft Corporation,Packaged software,Technology services,NASDAQ,"S&P 500, NASDAQ 100, Dow Jones Industrial Aver...",3.171456e+12,USD,426.59,USD,...,426.773058,US5949181045,0.860989,2024-07-30,2024-10-30,Strong buy,496.952444,USD,Strong sell,Titans
3,GOOGL,Alphabet Inc.,Internet software/Services,Technology services,NASDAQ,"S&P 500, NASDAQ 100, NASDAQ Composite, S&P 500...",2.062014e+12,USD,166.72,USD,...,166.704687,US02079K3059,1.648845,2024-07-23,2024-10-29,Buy,202.105263,USD,Sell,Titans
4,GOOG,Alphabet Inc.,Internet software/Services,Technology services,NASDAQ,"S&P 500, NASDAQ 100, NASDAQ Composite, S&P 500...",2.060605e+12,USD,168.34,USD,...,168.351993,US02079K1079,1.649972,2024-07-23,2024-10-29,Buy,201.928571,USD,Sell,Titans


In [ ]:
# # execute categorization
# category_setup_df = categorize_market_cap(pmgus_df).copy()
# # # drop Undefined marketCapType
# # category_setup_df = category_setup_df[category_setup_df['marketCapType'] != 'Undefined']

# # convert necessary columns to numeric
# def convert_columns_to_numeric(df, columns):
#     """Convert specified columns to numeric types."""
#     for col in columns:
#         df[col] = pd.to_numeric(df[col], errors='coerce')
#     return df

# # list of columns to convert
# numeric_columns = [
#     'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
#     'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
#     'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
#     'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume'
# ]

# # apply conversion
# category_setup_df = convert_columns_to_numeric(category_setup_df, numeric_columns)

In [ ]:
# Criteria configuration for each market cap category
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1_000_000_000,  # 1 billion shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.03,
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        "pre_market_volume_threshold": 50_000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    # "Midlers" in TradingView
    "Midlers": { 
        "pre_market_change_pct_threshold": 0.02,  # 2% for Midlers 
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

def filter_stocks(df, config):
    """Filter stocks based on configuration criteria."""
    conditions = (
        (df['Pre-market Change %'] >= config.get('pre_market_change_pct_threshold', 0)) &
        (df['Float shares outstanding'] <= config.get('float_shares_outstanding_threshold', float('inf'))) &
        (df['Relative Volume 1 day'] >= config.get('relative_volume_threshold', 0)) &
        (df['Relative Volume at Time'] >= config.get('relative_volume_at_time_threshold', 0)) &
        (df['Pre-market Gap %'] >= config.get('pre_market_gap_percentage_threshold', 0)) &
        (df['Price'] >= df['Volume Weighted Average Price 1 day'] * (1 - config.get('pre_market_vwap_drawdown_threshold', 0))) &
        (df['Volatility 1 day'] >= df['Volatility 1 week']) &
        (df['Volatility 1 day'] >= df['Volatility 1 month']) &
        (df['Pre-market Volume'] >= config.get('pre_market_volume_threshold', 0))
    )
    return df[conditions]

In [ ]:
def screen_stocks_by_category(df, category):
    """Filter stocks in a category using predefined criteria."""
    config = criteria_config.get(category, {})
    filtered_df = filter_stocks(df, config)
    return filtered_df

# execute filtering
smash_df = pd.DataFrame()
categories = category_setup_df['marketCapType'].unique()

for category in categories:
    category_df = category_setup_df[category_setup_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    smash_df = pd.concat([smash_df, gap_up_stage_df], ignore_index=True)

# add 'Market capitalization' to the reordered columns list
cols_list = [
    'Symbol', 'Description', 'Pre-market Change %', 'Pre-market Gap %', 'marketCapType',
    'Market capitalization','Price', 'Pre-market Open', 'Industry', 'Index', 'Sector', 
    'Exchange','Recent earnings date', 'Upcoming earnings date', 'Float shares outstanding', 
    'Average Volume 10 days', 'Average Volume 30 days', 'Average Volume 90 days',
    'Relative Volume 1 day', 'Relative Volume 5 minutes', 'Relative Volume 30 minutes', 
    'Relative Volume at Time'
]

# filter columns to only include those present in the DataFrame
existing_cols = [col for col in cols_list if col in smash_df.columns]
smash_df = smash_df[existing_cols]

# sort and reset index
smash_df = smash_df.sort_values(
    by=['Market capitalization', 'Pre-market Change %'],
    ascending=[False, False]).reset_index(drop=True)

print(f"{smash_df.shape[0]} stockes found in the screener.")


## ISSUE ##
#### HARD CODED FILE PATH ####
# also change this if you add another level, you probably will to screener_smoke 
# output_filename = f'pre_market_gap_up_screener_targets_{today_date_str}.csv'
output_filename = f'/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/tv_screen_gap-up_2024-10-28_NEW.csv'

smash_df.to_csv(output_filename, index=False)